In [1]:
!pip install -q transformers datasets scikit-learn accelerate torch pandas

import random
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import os

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    set_seed,
)
from datasets import load_dataset
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight

SEED = 42
MODEL_NAME = "microsoft/deberta-v3-large"
OUTPUT_DIR = "./deberta_task1_single"
MAX_LEN = 512
BATCH_SIZE = 4
GRAD_ACCUMULATION = 4
LR = 8e-6
EPOCHS = 15

def set_deterministic(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_deterministic(SEED)
set_seed(SEED)

dataset = load_dataset("ailsntua/QEvasion")

def preprocess(example):
    text = f"Question: {example['question']} Answer: {example['interview_answer']}"
    return {"text": text, "clarity_label": example["clarity_label"]}

full_train = dataset["train"].map(preprocess)
held_out_test_ds = dataset["test"].map(preprocess)
full_train = full_train.class_encode_column("clarity_label")
held_out_test_ds = held_out_test_ds.class_encode_column("clarity_label")

split = full_train.train_test_split(
    test_size=0.1,
    seed=SEED,
    stratify_by_column="clarity_label",
)
train_ds = split["train"]
eval_ds = split["test"]

print(f"Dataset Sizes: Train: {len(train_ds)}, Dev: {len(eval_ds)}, Test: {len(held_out_test_ds)}")

labels = train_ds.features["clarity_label"].names
label2id = {name: i for i, name in enumerate(labels)}
id2label = {i: name for name, i in label2id.items()}
print(f"Task 1 Labels: {labels}")

y_train = train_ds["clarity_label"]
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train,
)
device = "cuda" if torch.cuda.is_available() else "cpu"
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": (preds == labels).mean(),
        "macro_f1": f1_score(labels, preds, average="macro"),
    }

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(
            weight=class_weights_tensor,
            label_smoothing=0.1,
        )
        loss = loss_fct(
            logits.view(-1, model.config.num_labels),
            labels.view(-1),
        )
        return (loss, outputs) if return_outputs else loss

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
    )


current_train_ds = train_ds.map(tokenize_fn, batched=True)
current_eval_ds = eval_ds.map(tokenize_fn, batched=True)
current_test_ds = held_out_test_ds.map(tokenize_fn, batched=True)
current_train_ds = current_train_ds.map(lambda x: {"labels": x["clarity_label"]})
current_eval_ds = current_eval_ds.map(lambda x: {"labels": x["clarity_label"]})
current_test_ds = current_test_ds.map(lambda x: {"labels": x["clarity_label"]})

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    gradient_accumulation_steps=GRAD_ACCUMULATION,
    num_train_epochs=EPOCHS,
    weight_decay=0.05,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    fp16=True,
    report_to="none",
    seed=SEED,
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=current_train_ds,
    eval_dataset=current_eval_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)],
)

trainer.train()

test_results = trainer.evaluate(current_test_ds)

print("Final macro-F1:", round(test_results["eval_macro_f1"], 4))
print("Final accuracy:", round(test_results["eval_accuracy"], 4))
print("Final loss:", round(test_results["eval_loss"], 4))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3448 [00:00<?, ? examples/s]

...

pytorch_model.bin:   0%|          | 0.00/874M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/874M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,No log,1.147692,0.576812,0.376133
2,No log,1.114032,0.536232,0.493122
3,1.146000,0.999196,0.652174,0.629563
4,1.146000,0.940310,0.695652,0.684450
5,1.146000,1.046324,0.669565,0.649505
6,0.870600,1.120988,0.649275,0.640127
7,0.870600,1.230535,0.698551,0.649995
8,0.640500,1.285336,0.669565,0.633248


Final macro-F1: 0.6344
Final accuracy: 0.6753
Final loss: 0.9591
